In [1]:
import numpy as np
import pandas as pd
import time
import yfinance as yf
import matbplotlib.pyplot as plt
import functions as f

%load_ext autoreloard
%autoreload 2

ModuleNotFoundError: No module named 'yfinance'